In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import struct
import glob
import os

from plotly.subplots import make_subplots
import plotly.graph_objects as go

# pd.set_option('display.max_rows', None)

In [3]:
## RAW DATA

joined_files = os.path.join("../New Dataset (static)/SAEJ 6 Brake Network 250K.csv")
joined_list = glob.glob(joined_files)
dataset = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)
dataset2 = (dataset.sort_values(by=['Time (abs)'])).reset_index(drop=True)
dataset2.rename(columns={"ID (hex)":"CAN_ID", "Time (abs)":"Timestamp", "Data (hex)":"Databytes"},inplace=True)
dataset2['CAN_ID'] = dataset2.CAN_ID.astype(str)
dataset2

,Bus,No,Timestamp,State,CAN_ID,DLC,Databytes,ASCII
0,USB-to-CAN V2 compact CAN-1,"15,683",00:00:30.955,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
1,USB-to-CAN V2 compact CAN-1,"15,684",00:00:30.956,E,CF00203,8,CD 00 00 FF FF FF FF FF,........
2,USB-to-CAN V2 compact CAN-1,"15,685",00:00:30.960,E,18F62621,8,47 01 FA 02 F8 2A FF 16,G....*..
3,USB-to-CAN V2 compact CAN-1,"15,686",00:00:30.961,E,18FDC40B,8,E0 08 FF F3 FF FF FF FF,........
4,USB-to-CAN V2 compact CAN-1,"15,687",00:00:30.966,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
...,...,...,...,...,...,...,...,...
9995,USB-to-CAN V2 compact CAN-1,"25,678",00:00:50.628,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
9996,USB-to-CAN V2 compact CAN-1,"25,679",00:00:50.636,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
9997,USB-to-CAN V2 compact CAN-1,"25,681",00:00:50.637,E,CF00300,8,FD 00 FF FF FF FF FF FF,........
9998,USB-to-CAN V2 compact CAN-1,"25,680",00:00:50.637,E,CF00203,8,CD 00 00 FF FF FF FF FF,........


In [4]:
mask = (dataset2['CAN_ID'].str.len() > 6)
dataset2 = dataset2.loc[mask]
dataset2

,Bus,No,Timestamp,State,CAN_ID,DLC,Databytes,ASCII
0,USB-to-CAN V2 compact CAN-1,"15,683",00:00:30.955,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
1,USB-to-CAN V2 compact CAN-1,"15,684",00:00:30.956,E,CF00203,8,CD 00 00 FF FF FF FF FF,........
2,USB-to-CAN V2 compact CAN-1,"15,685",00:00:30.960,E,18F62621,8,47 01 FA 02 F8 2A FF 16,G....*..
3,USB-to-CAN V2 compact CAN-1,"15,686",00:00:30.961,E,18FDC40B,8,E0 08 FF F3 FF FF FF FF,........
4,USB-to-CAN V2 compact CAN-1,"15,687",00:00:30.966,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
...,...,...,...,...,...,...,...,...
9995,USB-to-CAN V2 compact CAN-1,"25,678",00:00:50.628,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
9996,USB-to-CAN V2 compact CAN-1,"25,679",00:00:50.636,E,CF00400,8,F0 7D 7D 00 00 FF F3 FF,.}}.....
9997,USB-to-CAN V2 compact CAN-1,"25,681",00:00:50.637,E,CF00300,8,FD 00 FF FF FF FF FF FF,........
9998,USB-to-CAN V2 compact CAN-1,"25,680",00:00:50.637,E,CF00203,8,CD 00 00 FF FF FF FF FF,........


In [5]:
# GET PARAMETER GROUP NUMBER (PGNs) FROM J1939-EXTENDED CAN ID 29-bit 

dataset2['PGN_[HEX]'] = dataset2['CAN_ID'].str[2] + dataset2['CAN_ID'].str[3] + dataset2['CAN_ID'].str[4] + dataset2['CAN_ID'].str[5]
dataset2['PGN_[DEC]'] = (dataset2['PGN_[HEX]']).apply(int, base=16)
dataset3 = pd.DataFrame(dataset2, columns=['Timestamp','CAN_ID','PGN_[HEX]', 'PGN_[DEC]','DLC','Databytes'])
dataset3.head()

,Timestamp,CAN_ID,PGN_[HEX],PGN_[DEC],DLC,Databytes
0,00:00:30.955,CF00400,F004,61444,8,F0 7D 7D 00 00 FF F3 FF
1,00:00:30.956,CF00203,F002,61442,8,CD 00 00 FF FF FF FF FF
2,00:00:30.960,18F62621,F626,63014,8,47 01 FA 02 F8 2A FF 16
3,00:00:30.961,18FDC40B,FDC4,64964,8,E0 08 FF F3 FF FF FF FF
4,00:00:30.966,CF00400,F004,61444,8,F0 7D 7D 00 00 FF F3 FF


In [6]:
# IMPORT J1939 DATABASE CAN (DBC)

dbc_dataframe = pd.read_csv('../New Dataset (static)/0 j1939_dbc.csv', sep = ';')
dbc_dataframe

,Name,ID,ID-Format,DLC [Byte],Tx Method,Cycle Time,Transmitter,Comment,PGN,Source,Destination,Priority,GenMsgStartDelayTime,GenMsgILSupport,GenMsgDelayTime,GenMsgRequestable,GenMsgSendType,GenMsgCycleTime
0,A1,0x18FEFDFE,J1939 PG (ext. ID),8.0,cyclic,500.0,-- No Transmitter --,Alternate Fuel 1,0xFEFD,NaN,All,0x6,0.0,Yes,0.0,1.0,cyclic,500.0
1,A1DOC,0x18FD20FE,J1939 PG (ext. ID),8.0,cyclic,500.0,-- No Transmitter --,Aftertreatment 1 Diesel Oxidation Catalyst,0xFD20,NaN,All,0x6,0.0,Yes,0.0,1.0,cyclic,500.0
2,A1GOC,0x18FD22FE,J1939 PG (ext. ID),8.0,cyclic,500.0,-- No Transmitter --,Aftertreatment 1 Gas Oxidation Catalyst,0xFD22,NaN,All,0x6,0.0,Yes,0.0,1.0,cyclic,500.0
3,A1SCRAI,0xCF025FE,J1939 PG (ext. ID),8.0,cyclic,50.0,-- No Transmitter --,Aftertreatment 1 SCR Ammonia Information,0xF025,NaN,All,0x3,0.0,Yes,0.0,1.0,cyclic,50.0
4,A1SCRDSI1,0xCF023FE,J1939 PG (ext. ID),8.0,cyclic,50.0,-- No Transmitter --,Aftertreatment 1 SCR Dosing System Information 1,0xF023,NaN,All,0x3,0.0,Yes,0.0,1.0,cyclic,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,NaN,0x18FF20FE,J1939 PG (ext. ID),NaN,NaN,NaN,NaN,Proprietary,0xFF20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
551,NaN,0x18FF01FE,J1939 PG (ext. ID),NaN,NaN,NaN,NaN,Proprietary,0xFF01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
552,NaN,0x181D00FE,J1939 PG (ext. ID),NaN,NaN,NaN,NaN,Engine Ignition Controller,0x1D00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
553,NaN,0x18F310FE,J1939 PG (ext. ID),NaN,NaN,NaN,NaN,High Voltage Energy Storage Pack 23 Data 8,0xF310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
mask = dbc_dataframe['PGN'].str.len() == 5
df = dbc_dataframe.loc[mask]
df

,Name,ID,ID-Format,DLC [Byte],Tx Method,Cycle Time,Transmitter,Comment,PGN,Source,Destination,Priority,GenMsgStartDelayTime,GenMsgILSupport,GenMsgDelayTime,GenMsgRequestable,GenMsgSendType,GenMsgCycleTime


In [8]:
# GET PGNs LABEL FROM DBC

pd.set_option('display.max_rows', None)
dbc_dataframe['PGN'] = dbc_dataframe.PGN.astype(str)
dbc_dataframe['PGN_ID'] = dbc_dataframe['PGN'].str[2] + dbc_dataframe['PGN'].str[3] + dbc_dataframe['PGN'].str[4] + dbc_dataframe['PGN'].str[5]
dbc_dataframe['PF'] = dbc_dataframe['PGN'].str[2] + dbc_dataframe['PGN'].str[3]
dbc_dataframe['PS'] = dbc_dataframe['PGN'].str[4] + dbc_dataframe['PGN'].str[5]

dbc_dataframe['PGN_ID_DEC'] = dbc_dataframe['PGN'].apply(int, base=16)

PGN_conv = pd.DataFrame(dbc_dataframe, columns=['PGN_ID', 'PGN_ID_DEC','PF','PS','Comment'])
PGN_conv

,PGN_ID,PGN_ID_DEC,PF,PS,Comment
0,FEFD,65277,FE,FD,Alternate Fuel 1
1,FD20,64800,FD,20,Aftertreatment 1 Diesel Oxidation Catalyst
2,FD22,64802,FD,22,Aftertreatment 1 Gas Oxidation Catalyst
3,F025,61477,F0,25,Aftertreatment 1 SCR Ammonia Information
4,F023,61475,F0,23,Aftertreatment 1 SCR Dosing System Information 1
5,FD41,64833,FD,41,Aftertreatment 1 SCR Dosing System Information 2
6,F024,61476,F0,24,Aftertreatment 1 SCR Dosing System Requests 1
7,FD40,64832,FD,40,Aftertreatment 1 SCR Dosing System Requests 2
8,FD3F,64831,FD,3F,Aftertreatment 1 SCR Exhaust Gas Pressures
9,FD3E,64830,FD,3E,Aftertreatment 1 SCR Exhaust Gas Temperature


In [9]:
# GET A LIST OF DETECTED PGNs

PGN_List = pd.DataFrame(dataset3, columns=['PGN_[HEX]','CAN_ID'])
PGN_List['PF'] = PGN_List['PGN_[HEX]'].str[0] + PGN_List['PGN_[HEX]'].str[1]
PGN_List['PS'] = PGN_List['PGN_[HEX]'].str[2] + PGN_List['PGN_[HEX]'].str[3]
PGN_List['PGN_LABEL'] = PGN_List['PGN_[HEX]'].apply(int, base=16)
PGN_List = (PGN_List.sort_values(by=['PGN_[HEX]'])).reset_index(drop=True)
PGN_List = PGN_List.drop_duplicates()
PGN_List = PGN_List.reset_index(drop=True)

for ind in PGN_List.index:
    for ins in PGN_conv.index:
        if PGN_List['PGN_[HEX]'][ind] == PGN_conv['PGN_ID'][ins]:
            PGN_List = PGN_List.replace([PGN_List['PGN_LABEL'][ind]],PGN_conv['Comment'][ins])
            break
            
        elif (PGN_List['PF'][ind] == PGN_conv['PF'][ins]) and (PGN_conv['PS'][ins] == '00'):
            PGN_List = PGN_List.replace([PGN_List['PGN_LABEL'][ind]],PGN_conv['Comment'][ins])

PGN_List

,PGN_[HEX],CAN_ID,PF,PS,PGN_LABEL
0,0000,C00000B,00,00,Torque/Speed Control 1
1,A9E8,18A9E827,A9,E8,Forward Lane Image Command
2,EBFF,18EBFFEF,EB,FF,Transport Protocol - Data Transfer
3,EBFF,18EBFF00,EB,FF,Transport Protocol - Data Transfer
4,EBFF,18EBFF10,EB,FF,Transport Protocol - Data Transfer
5,ECFF,18ECFF10,EC,FF,Transport Protocol - Connection Management
6,ECFF,18ECFFEF,EC,FF,Transport Protocol - Connection Management
7,ECFF,18ECFF00,EC,FF,Transport Protocol - Connection Management
8,EFEF,CEFEF0B,EF,EF,Proprietary
9,F000,18F00010,F0,00,Electronic Transmission Controller 1


In [43]:
df_EBC = dataset3[dataset3["PGN_[HEX]"] == 'F001']
df_EBC['Data'] = df_EBC['Databytes'].str.split(',') 
df_EBC

C:\Users\PRIET\AppData\Local\Temp\ipykernel_17052\3186242912.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_EBC['Data'] = df_EBC['Databytes'].str.split(',')


,Timestamp,CAN_ID,PGN_[HEX],PGN_[DEC],DLC,Databytes,Data
16,00:00:30.990,18F00117,F001,61441,8,FF FF FF FF FF FF FF F7,[FF FF FF FF FF FF FF F7]
51,00:00:31.048,18F0010B,F001,61441,8,00 00 F3 FF FF 01 FF F7,[00 00 F3 FF FF 01 FF F7]
69,00:00:31.089,18F00117,F001,61441,8,FF FF FF FF FF FF FF F7,[FF FF FF FF FF FF FF F7]
99,00:00:31.148,18F0010B,F001,61441,8,00 00 F3 FF FF 01 FF F7,[00 00 F3 FF FF 01 FF F7]
118,00:00:31.189,18F00117,F001,61441,8,FF FF FF FF FF FF FF F7,[FF FF FF FF FF FF FF F7]
150,00:00:31.249,18F0010B,F001,61441,8,00 00 F3 FF FF 01 FF F7,[00 00 F3 FF FF 01 FF F7]
168,00:00:31.290,18F00117,F001,61441,8,FF FF FF FF FF FF FF F7,[FF FF FF FF FF FF FF F7]
201,00:00:31.349,18F0010B,F001,61441,8,00 00 F3 FF FF 01 FF F7,[00 00 F3 FF FF 01 FF F7]
220,00:00:31.390,18F00117,F001,61441,8,FF FF FF FF FF FF FF F7,[FF FF FF FF FF FF FF F7]
251,00:00:31.448,18F0010B,F001,61441,8,00 00 F3 FF FF 01 FF F7,[00 00 F3 FF FF 01 FF F7]
